In [2]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2
import os
from utils.path import dir_HugeFiles
from utils.preprocessing import load, preprocessing
from utils.words import clean_wordcount, replace_UNK, make_corpus, get_wordcount, make_corpus_0
from utils.save import make_dir, save_pickle, load_pickle
from utils.word2vec import train_wordvec
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.77 µs


In [19]:
dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
dic = load(dir_save)

exist
time: 16.2 s


In [6]:
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))

drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
time: 168 ms


### prepare input of bert 

In [23]:
# load amt
amt_hGI_pos = load_pickle('data/amt_hGI_pos.pickle')

# add amt tags
for i, v in dic.items():
    if i in amt_hGI_pos.keys():
        v['UD'] = amt_hGI_pos[i] 
ls = list(amt_hGI_pos.keys())

time: 6.54 s


In [24]:
df = pd.DataFrame.from_dict(dic, orient = 'index')
df = df.iloc[ls].reset_index()[['index','name_UNK','ingredients_UNK','directions_UNK','UD']]
df['name_UNK'] = df['name_UNK'].apply(lambda x: x[0]+' , ')
df['ingredients_UNK'] = df['ingredients_UNK'].apply(lambda x: ' , '.join(x))
df['directions_UNK'] = df['directions_UNK'].apply(lambda x: ' '.join(x))
df['all'] = df['name_UNK']+df['ingredients_UNK']+df['directions_UNK']
df['HD'] = (df.UD == 0).astype(int)
#df['index'] = df['index'].apply(lambda x: '"{}"'.format(x))
df['index'] = df['index'].astype(str)

time: 2.22 s


In [25]:
train_bert = df[['index','all','UD','HD']].rename({'index':'id', 'all': 'comment_text'}, axis = 1)
#train_bert.to_csv(dir_HugeFiles+'/bert_data/dataset/raw/train.csv', index =False)

time: 45.5 ms


In [26]:
train_bert.head()

,id,comment_text,UD,HD
0,18181,"brazilian black bean stew , 1 tablespoon canol...",1.0,0
1,25569,japanese country style miso and tofu UNK UNK U...,0.0,1
2,1136,"outer UNK style clam dish , 1/3 pound fatback ...",1.0,0
3,24707,"salsa rosa , 3 tablespoons mayonnaise , 1 teas...",0.0,1
4,23245,"creamy artichoke pasta , 12 ounces uncooked sp...",1.0,0


time: 47.6 ms


In [27]:
from sklearn.model_selection import train_test_split

time: 35.3 ms


In [33]:
# load amt
amt_hGI_pos = load_pickle('data/amt_hGI_pos.pickle')

# add amt tags
for i, v in dic.items():
    if i in amt_hGI_pos.keys():
        v['UD'] = amt_hGI_pos[i] 
ls = list(amt_hGI_pos.keys())

df = pd.DataFrame.from_dict(dic, orient = 'index')
df = df.iloc[ls].reset_index()[['index','name_UNK','ingredients_UNK','directions_UNK','UD']]
df['name_UNK'] = df['name_UNK'].apply(lambda x: x[0]+' , ')
df['ingredients_UNK'] = df['ingredients_UNK'].apply(lambda x: ' , '.join(x))
df['directions_UNK'] = df['directions_UNK'].apply(lambda x: ' '.join(x))
df['all'] = df['name_UNK']+df['ingredients_UNK']+df['directions_UNK']
df['index'] = df['index'].astype(str)
train_bert = df[['index','all','UD']].rename({'index':'id', 'all': 'comment_text'}, axis = 1)
X_train, X_test, y_train, y_test = train_test_split(train_bert.index.tolist(),train_bert.UD, stratify = train_bert.UD)

train_bert.iloc[X_train].to_csv(dir_HugeFiles+'bert_0611/dataset/raw/train.csv', index =False)
train_bert.iloc[X_test].to_csv(dir_HugeFiles+'bert_0611/dataset/raw/test.csv', index =False)

time: 8.98 s


In [32]:
# pos weight
pos = sum(y_train)
total = len(y_train)
neg = total- pos
neg/pos

0.9577836411609498

time: 39.8 ms


In [35]:
from utils.save import make_dir
# load amt
amt_hGI_pos = load_pickle('data/amt_hGI_pos.pickle')

# add amt tags
for i, v in dic.items():
    if i in amt_hGI_pos.keys():
        v['UD'] = amt_hGI_pos[i] 
ls = list(amt_hGI_pos.keys())

df = pd.DataFrame.from_dict(dic, orient = 'index')
df = df.iloc[ls].reset_index()[['index','name_UNK','ingredients_UNK','directions_UNK','UD']]
df['name_UNK'] = df['name_UNK'].apply(lambda x: x[0]+' , ')
df['ingredients_UNK'] = df['ingredients_UNK'].apply(lambda x: ' , '.join(x))
df['directions_UNK'] = df['directions_UNK'].apply(lambda x: ' '.join(x))
df['all'] = df['name_UNK']+df['ingredients_UNK']+df['directions_UNK']
df['index'] = df['index'].astype(str)
train_bert = df[['index','all','UD']].rename({'index':'id', 'all': 'comment_text'}, axis = 1)
X_train, X_test, y_train, y_test = train_test_split(train_bert.index.tolist(),train_bert.UD, stratify = train_bert.UD)

path = dir_HugeFiles+'bert_0612/'
make_dir(path+'dataset/raw/')
train_bert.iloc[X_train].to_csv(path + 'dataset/raw/train.csv', index = False)
train_bert.iloc[X_test].to_csv(path + 'dataset/raw/test.csv', index = False)

make_dir(path+'dataset/processed/')
make_dir(path+'model/')
make_dir(path+'output/checkpoints/')
make_dir(path+'output/figure/')
make_dir(path+'output/log/')

make dir
make dir
make dir
time: 8.69 s
